<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>

# Google GenAI SDK - Building a Sequential Agent

## Install Dependencies

In [ ]:
!pip install -q google-genai arize-phoenix-otel openinference-instrumentation-google-genai

## Connect to Arize Phoenix

In [ ]:
import os
from getpass import getpass

from google import genai

from phoenix.otel import register

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass("Enter your Phoenix API key: ")

os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com/"

tracer_provider = register(auto_instrument=True, project_name="google-genai-sequential-agent")
tracer = tracer_provider.get_tracer(__name__)

## Authenticate with Google Vertex AI

In [ ]:
!gcloud auth login

In [ ]:
# Create a client using the Vertex AI API, you could also use the Google GenAI API instead here
client = genai.Client(vertexai=True, project="<ADD YOUR GCP PROJECT ID>", location="us-central1")

# Sequential Agent

In [ ]:
# --- 1. Define helper methods ---


def get_model(model_name):
    """Instantiate and return a model."""
    return model_name


@tracer.chain
def research_topic(client, model, user_input):
    """Research a topic based on user input."""
    research_prompt = (
        "You are a research assistant.\n"
        "Provide a comprehensive overview of the following topic.\n"
        "Include key facts, historical context, and current relevance.\n"
        "Keep your response to 3-4 paragraphs.\n"
        "User Input: "
    )
    research_response = client.models.generate_content(
        model=model,
        contents=research_prompt + user_input,
    )
    return research_response.text.strip()


@tracer.chain
def identify_key_points(client, model, research):
    """Extract key points from the research."""
    key_points_prompt = (
        "You are a research summarizer.\n"
        "Extract 5-7 key points from the following research.\n"
        "Format each point as a bullet point with a brief explanation.\n\n"
        f"Research:\n{research}"
    )
    key_points_response = client.models.generate_content(model=model, contents=key_points_prompt)
    return key_points_response.text.strip()


@tracer.chain
def generate_future_directions(client, model, research, key_points):
    """Generate future research directions based on the research and key points."""
    future_prompt = (
        "You are a research strategist.\n"
        "Based on the research and key points provided, suggest 3-4 promising future research directions.\n"
        "For each direction, explain its potential significance and impact.\n\n"
        f"Research:\n{research}\n\n"
        f"Key Points:\n{key_points}\n\n"
    )
    future_response = client.models.generate_content(model=model, contents=future_prompt)
    return future_response.text.strip()


# --- 2. Main execution ---
@tracer.agent()
def run_agent(user_input):
    # Instantiate the models
    research_model = get_model("gemini-2.0-flash-001")

    # Step 1: Generate the initial research
    initial_research = research_topic(client, research_model, user_input)

    # Step 2: Identify key points from the research
    extracted_key_points = identify_key_points(client, research_model, initial_research)

    # Step 3: Generate future research directions
    future_directions = generate_future_directions(
        client, research_model, initial_research, extracted_key_points
    )

    # Display the results
    print("=== Initial Research ===\n")
    print(initial_research)
    print("\n=== Key Points ===\n")
    print(extracted_key_points)
    print("\n=== Future Research Directions ===\n")
    print(future_directions)

    return {
        "research": initial_research,
        "key_points": extracted_key_points,
        "future_directions": future_directions,
    }


run_agent(user_input=input("Please enter a topic you'd like to research: "))